# Setup

In [1]:
from collections import defaultdict
from datetime import datetime
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from cost import *
from data import *
from energy import *
from inflation import *
from parameters import *
from plotting import *
from utils import *

In [2]:
results_dir = f'results/development-costs/'
os.makedirs(results_dir, exist_ok=True)

Sample size for distributions

In [3]:
N = 1000

Cost of staff compensation
- Use Google Software Engineer as the reference: https://web.archive.org/web/20240515221234/https://www.levels.fyi/companies/google/salaries/software-engineer/locations/san-francisco-bay-area?dma=807
- Approximately 200K to 1.2M


In [4]:
compensation_low = 2e5
compensation_high = 1.2e6


Apply 1.25 - 1.4 overhead factor for benefits, payroll, etc. https://www.sba.gov/blog/how-much-does-employee-cost-you 

In [5]:
compensation_overhead_low = 1.25
compensation_overhead_high = 1.4

Anchor number of FTEs to the author count on the paper.
For models we consider here, the author list is a decent anchor point for the actual contributors, because e.g. engineers are included.

In [6]:
author_factor_low = 0.3
author_factor_high = 1.5

In [7]:
cost_data = {}
cost_data_with_hardware_acq = {}

Default ratio of total experiment + training compute to final training run compute
- Smaller versions of GPT-3 made up ~4.5e22 FLOP (based on the formula compute = 6 * parameters * tokens), compared to 175B using ~3e23 FLOP. See Table 2.1 of the [paper](https://arxiv.org/pdf/2005.14165). That's a factor of ~1.14 to go from 175B to total FLOP. Certainly more than that was needed due to failures and other experiments. So 1.2x seems like a good low-end estimate.
- https://arxiv.org/pdf/2111.00364.pdf, p.3, Fig. 3 caption: "At Facebook, we observe a rough power capacity breakdown of 10:20:70 for AI infrastructures devoted to the three key phases — Experimentation, Training, and Inference". 10:20 ratio means a 1.5x multiplier from training to training+experiment compute. Note however that the "training" category is not just the final training run; it includes additional hyper-parameter tuning and retraining for some models. So the actual multiplier would be higher under our definition.
- BLOOM project - https://arxiv.org/abs/2211.02001, p.8, Table 5. ~63% of compute was spent training smaller models than 176B. That's a multiplier of ~2.7x.
- We think the Facebook and BLOOM cases are more central examples as they account better for all experiments. A factor close to 2x seems like a reasonable median estimate.
- On the high end, it's plausible that several, full iterations of the full-scale training run are necessary before success - say, 5x.

In [8]:
experiment_factor = lognorm_from_90_ci(1.2, 5, N)
print_median_and_ci(experiment_factor)

Median: 2.5 [90% CI: 1.2, 5.1]


Uncertainty in energy prices

In [9]:
likely_datacenter_states = ['California', 'Nevada', 'Oregon', 'Washington']
energy_prices = [US_STATE_ENERGY_PRICES_PER_KWH[state] for state in likely_datacenter_states]
mean = np.mean(energy_prices)
np.max(energy_prices) / mean, np.min(energy_prices) / mean

(1.727914785226352, 0.6247539654972791)

In [10]:
energy_price_uncertainty_factor = lognorm_from_90_ci(0.6, 1.7, N)

In [11]:
dt2float = lambda x: x.year + x.month/12 + x.day/365.25
float2dt = lambda x: datetime(int(x), int((x-int(x))*12), int(((x-int(x))*12-int((x-int(x))*12))*365.25))

In [12]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation(
    compute_threshold_method='top_n', compute_threshold=10,
)

Models

In [13]:
models = ['GPT-3 175B (davinci)', 'OPT-175B', 'GPT-4', 'Gemini Ultra']

In [14]:
selected_model_df = frontier_pcd_df.loc[[s in models for s in frontier_pcd_df['System']]]

## Hardware

In [15]:
# Unset index
selected_model_df.reset_index(inplace=True)

In [16]:
def _estimate_chip_hours(row):
    return estimate_chip_hours(row, hardware_df)

selected_model_df['Training chip-hours'] = selected_model_df.apply(_estimate_chip_hours, axis=1)
selected_model_df['Training chip-hours']

/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/ipykernel_72042/1294256109.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_model_df['Training chip-hours'] = selected_model_df.apply(_estimate_chip_hours, axis=1)


0    132000000.0
1     57000000.0
2       812544.0
3      3552000.0
Name: Training chip-hours, dtype: float64

In [17]:
selected_model_df.loc[:, 'Original training chip-hours'] = selected_model_df.loc[:, 'Training chip-hours']
hardware_costs = defaultdict(dict)
cost_component_names = [
    'AI accelerator chip cost',
    'Other server components cost',
    'Cluster-level interconnect cost',
    'Energy cost',
]

for percentile in [2.5, 50, 97.5]:
    print(f'\n\n{percentile}th percentile of experiments time')
    experiment_factor_value = np.percentile(experiment_factor, percentile)
    selected_model_df.loc[:, 'Training chip-hours'] = selected_model_df.loc[:, 'Original training chip-hours'] * experiment_factor_value
    cost_df = estimate_hardware_capex_opex(selected_model_df, hardware_df, price_df, separate_components=True)
    cost_df = cost_df.set_index('System', inplace=False)

    for model in models:
        cost_components = {}
        for cost_component_name in cost_component_names:
            cost_components[cost_component_name] = cost_df.loc[model, cost_component_name]
        hardware_costs[model][str(percentile)] = cost_components

hardware_costs



2.5th percentile of experiments time
==== System: Gemini Ultra ====
Estimated the value of Google TPU v4 server, available from 2021-05-20 00:00:00 and used from 2023-05-10 00:00:00: 10527.742991314644

==== System: GPT-4 ====
Estimated the value of NVIDIA A100 SXM4 40 GB server, available from 2020-08-12 00:00:00 and used from 2022-05-12 00:00:00: 14165.054738257799

==== System: OPT-175B ====
Estimated the value of NVIDIA A100 SXM4 80 GB server, available from 2021-02-14 00:00:00 and used from 2022-02-27 23:00:00: 17818.700400260856

==== System: GPT-3 175B (davinci) ====
Estimated the value of NVIDIA Tesla V100 DGXS 32 GB server, available from 2018-06-25 00:00:00 and used from 2019-10-01 00:00:00: 11463.61551009175

==== System: Gemini Ultra ====
Estimated cost: {'AI accelerator chip cost': 32870212.285350483, 'Other server components cost': 21036935.862624306, 'Cluster-level interconnect cost': 8086072.222196214, 'Energy cost': 3088861.5127483187}
==== System: GPT-4 ====
Estimat

/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/ipykernel_72042/3550706096.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_model_df.loc[:, 'Original training chip-hours'] = selected_model_df.loc[:, 'Training chip-hours']
/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/ipykernel_72042/3550706096.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_model_df.loc[:, 'Training chip-hours'] = selected_model_df.loc[:, 'Original training chip-hours'] * experiment_factor_value
/Users/bencotti

defaultdict(dict,
            {'GPT-3 175B (davinci)': {'2.5': {'AI accelerator chip cost': 934641.4026206905,
               'Other server components cost': 644902.5678082764,
               'Cluster-level interconnect cost': 236931.5955643449,
               'Energy cost': 117148.02339067386},
              '50': {'AI accelerator chip cost': 2171293.3242681543,
               'Other server components cost': 1498192.393745026,
               'Cluster-level interconnect cost': 550422.8577019768,
               'Energy cost': 272150.0678507914},
              '97.5': {'AI accelerator chip cost': 5276236.984818345,
               'Other server components cost': 3640603.519524658,
               'Cluster-level interconnect cost': 1337526.0756514496,
               'Energy cost': 661323.9387631585}},
             'OPT-175B': {'2.5': {'AI accelerator chip cost': 338338.9342792743,
               'Other server components cost': 223303.69662432105,
               'Cluster-level interconnect c

# GPT-3

## Project duration

Training end time
1. Shevlane (2022)
https://uploads-ssl.webflow.com/614b70a71b9f71c9c240c7a7/6262a1a55526a373cc93207d_Shevlane%20dissertation%20preprint.pdf
p.66 of the PDF: A senior member of OpenAI (specified anonymously on p.27 of the PDF) 
told the author "GPT-3 existed for a long time before the paper came out. We delayed the 
paper. [...] But it's months, it doesn't really count."
p.67 of the PDF: CAMERON said "Firstly, [the idea for a commercial API for GPT-3] 
started out as a research API. It probably was . . . early January 2020."

1. We think it plausibly could have been produced soon after the Microsoft deal was 
announced in July 2019. Supposing the announcement coincided with Microsoft giving 
OpenAI access to the necessary compute, and OpenAI already having almost everything 
about GPT-3 planned in advance, and it took less than 1 month to train, then GPT-3 could 
have been produced in August 2019.

1. So we estimate August to January as our 90% CI, and halfway between (October) as the
central estimate.

In [18]:
training_end_low = datetime(2019, 8, 1)
training_end_high = datetime(2020, 1, 1)


Project start time
- Lower bound on start time: when GPT-2 was published, 2019-Feb-14
  - It could have been earlier than this, but that seems unlikely unless GPT-2 was delayed for months
  - The scaling laws paper was only published in January 2020, and GPT-3 was based on the findings of that paper
- Upper bound on start time: 2019-Nov-01, two months before the upper bound on training completion
  - We think they’d need six weeks at the absolute minimum, to prepare training data and scale up the experiments successfully


In [19]:
project_start_low = datetime(2019, 2, 14)
project_start_high = datetime(2019, 11, 1)


In [20]:
frontier_pcd_df['Publication date'] = pd.to_datetime(frontier_pcd_df['Publication date'])

In [21]:
publication_date = frontier_pcd_df.loc[frontier_pcd_df['System'] == 'GPT-3 175B (davinci)', 'Publication date'].values[0]
publication_date

numpy.datetime64('2020-05-28T00:00:00.000000000')

In [22]:
min_project_duration = 15 / 365  # training duration
experiments_duration = []
while len(experiments_duration) < N:
    project_start_sample = lognorm_from_90_ci(dt2float(project_start_low), dt2float(project_start_high), 1)[0]
    training_end_sample = lognorm_from_90_ci(dt2float(training_end_low), dt2float(training_end_high), 1)[0]
    experiment_duration_sample = training_end_sample - project_start_sample
    if experiment_duration_sample > min_project_duration:
        experiments_duration.append(experiment_duration_sample)

project_duration = []
while len(project_duration) < N:
    project_start_sample = lognorm_from_90_ci(dt2float(project_start_low), dt2float(project_start_high), 1)[0]
    # Paper was delayed by "months"
    publication_delay_sample = lognorm_from_90_ci(2*30, 4*30, 1)[0]
    project_end_sample = dt2float(publication_date - pd.to_timedelta(publication_delay_sample, unit='D'))
    project_duration_sample = project_end_sample - project_start_sample
    if project_duration_sample > min_project_duration:
        project_duration.append(project_duration_sample)

experiments_duration = np.array(experiments_duration)
project_duration = np.array(project_duration)
print_median_and_ci(experiments_duration * 12)  # convert years to months
print_median_and_ci(project_duration * 12)

Median: 4.3 [90% CI: 1, 9.2]
Median: 8.2 [90% CI: 3.8, 13]


## Energy

- Total energy consumption of training: 1287 MWh (Patterson et al. 2021)

In [23]:
energy_consumption_kwh = 1287e3 * experiment_factor
price_per_kwh = energy_price(2020) * energy_price_uncertainty_factor
energy_cost = energy_consumption_kwh * price_per_kwh
print_median_and_ci(energy_cost)

Median: 2.1e+05 [90% CI: 8.4e+04, 5e+05]


## Personnel

In [24]:
total_salaries = np.zeros(N)
num_authors = 31
for i in range(N):
    ftes = int(lognorm_from_90_ci(author_factor_low * num_authors, author_factor_high * num_authors, 1)[0])
    salary = lognorm_from_90_ci(compensation_low, compensation_high, ftes)
    total_salaries[i] = salary.sum()

In [25]:
personnel_cost = total_salaries * project_duration

In [26]:
print_median_and_ci(personnel_cost, ci=[2.5, 97.5])

Median: 7.5e+06 [95% CI: 1.8e+06, 2.4e+07]


## Overall

In [27]:
hardware_costs['GPT-3 175B (davinci)']['2.5']

{'AI accelerator chip cost': 934641.4026206905,
 'Other server components cost': 644902.5678082764,
 'Cluster-level interconnect cost': 236931.5955643449,
 'Energy cost': 117148.02339067386}

In [28]:
def get_hardware_cost_dist(model, component, ci=[2.5, 97.5]):
    return lognorm_from_ci(
        hardware_costs[model][str(ci[0])][component],
        hardware_costs[model][str(ci[1])][component],
        ci[1] - ci[0],
        N
    )

In [29]:
cost_data['GPT-3 175B (davinci)'] = {
    'AI accelerator chip cost': get_hardware_cost_dist('GPT-3 175B (davinci)', 'AI accelerator chip cost'),
    'Other server components cost': get_hardware_cost_dist('GPT-3 175B (davinci)', 'Other server components cost'),
    'Cluster-level interconnect cost': get_hardware_cost_dist('GPT-3 175B (davinci)', 'Cluster-level interconnect cost'),
    'Energy cost': energy_cost,
    'R&D staff cost': personnel_cost,
}

# OPT-175B

## GPU hardware

$167000 for DGX A100, 8 GPU, 640GB. In Jan 2022. From a reseller.
https://web.archive.org/web/20220120191032/https://www.dihuni.com/product/dgx-a100-comparable-dihuni-optiready-ai-a100-sxm4-8nve-8-x-ampere-a100-sxm4-nvlink-epyc-deep-learning-server/

In [30]:
num_gpus = 992
price_per_server = 167e3
gpus_per_server = 8
num_servers = num_gpus / gpus_per_server
gpu_hardware_cost = price_per_server * num_servers
printg(gpu_hardware_cost)

2.0708e+07


## Other hardware

The NVIDIA DGX A100 server already covers CPUs and node-level interconnect
We assume that the biggest cost on top of that is cluster-level interconnect (transceivers, switches, and cables). 
Assume $11 per Gbps of server-level interconnect
How fast is A100 interconnect? With NVLink it’s 600 Gbps per GPU (https://www.nvidia.com/en-gb/data-center/tesla-v100/).


In [31]:
cluster_interconnect_cost_per_gbps = 11
cluster_interconnect_bandwidth = 600
cluster_interconnect_cost = cluster_interconnect_cost_per_gbps * cluster_interconnect_bandwidth * num_gpus
printg(cluster_interconnect_cost)

6.5472e+06


## Energy

https://arxiv.org/abs/2205.01068: "[...] OPT-175B was developed with an estimated carbon emissions footprint (CO2eq) of 75 tons [...] With ablations, baselines and downtime, our own estimates of total cost is roughly 2× higher."

- I haven't found numbers for carbon intensity, or how they calculate it
- According to https://arxiv.org/pdf/2104.10350.pdf (Table 1), Gross CO2e/KWh (kg/KWh) for the “Google Iowa Council Bluffs” data center in 2020 was 0.478 CO2e/kWh (kg/kWh)
- Energy use:
  - 992 A100 GPU units
  - 300W per unit (https://www.nvidia.com/en-us/data-center/a100/)
  - Training time 33 days (https://github.com/facebookresearch/metaseq/blob/d703cf1ae1e0faaff6c20629398dfbe02b98cf77/projects/OPT/chronicles/final_update.md)
  - Energy = 992 units * 0.3 kW/unit * 33 days * 24h/day ~= 236000 kWh
  - 75000 kg / 236000 kWh ~= 0.318 CO2e/kWh (kg/kWh)
  - Sanity check: this is similar to the Google data center figure of 0.478

In [32]:
training_time = 33 * 24
power_per_gpu = 0.3
experiment_factor = 2.0
energy_consumption_kwh = training_time * num_gpus * power_per_gpu * experiment_factor
price_per_kwh = 0.083
energy_cost = energy_consumption_kwh * price_per_kwh
energy_cost

39126.0672

## Personnel

2021-Oct-20: First date in [OPT logbook](https://github.com/facebookresearch/metaseq/blob/main/projects/OPT/chronicles/OPT175B_Logbook.pdf)

2022-Jan-06: Date in the logbook where the final training run is stated to have completed

In [33]:
experiments_duration = 78 / DAYS_PER_YEAR

In [34]:
num_authors = 19

In [35]:
total_salaries = np.zeros(N)
for i in range(N):
    ftes = int(lognorm_from_90_ci(0.3 * num_authors, 1.5 * num_authors, 1)[0])
    salary = lognorm_from_90_ci(3e5, 2e6, ftes)
    total_salaries[i] = salary.sum()

In [36]:
personnel_cost = total_salaries * experiments_duration

In [37]:
print_median_and_ci(personnel_cost)

Median: 2.3e+06 [90% CI: 8.9e+05, 5.4e+06]


## Overall

In [38]:
cost_data_upfront_capex['OPT-175B'] = {
    'GPU servers': gpu_hardware_cost,
    'Other hardware': cluster_interconnect_cost,
    'Energy': energy_cost,
    'Personnel': personnel_cost.mean(),
}

NameError: name 'cost_data_upfront_capex' is not defined

In [ ]:
a100_server_lifetime = lognorm_from_90_ci(2, 7, N)  # years

In [ ]:
cost_data_amortized_capex['OPT-175B'] = {
    'GPU servers': np.mean(gpu_hardware_cost * experiments_duration / a100_server_lifetime),
    'Other hardware': np.mean(cluster_interconnect_cost * experiments_duration / a100_server_lifetime),
    'Energy': energy_cost,
    'Personnel': personnel_cost.mean(),
}
cost_data_amortized_capex['OPT-175B']

{'GPU servers': 1251889.3745022374,
 'Other hardware': 395806.9399623841,
 'Energy': 39126.0672,
 'Personnel': 2621423.5046858904}

# GPT-4

Use the [estimate by Yafah Edelman](https://www.lesswrong.com/posts/nXcHe7t4rqHMjhzau/report-on-frontier-model-training#Cost_Breakdown_of_ML_Training) as a placeholder

In [ ]:
total_cost = 5e8
cost_data_upfront_capex['GPT-4'] = {
    'GPU servers': 0.569 * total_cost,
    'Other hardware': 0.244 * total_cost,
    'Energy': 0.01 * total_cost,
    'Personnel': 0.178 * total_cost,
}

In [ ]:
# Amortized capex
upfront_hardware_capex = (0.569 + 0.244) * total_cost
training_time = 2280 / HOURS_PER_YEAR  # in years, from PCD database
experiment_factor = 2
amortized_server_capex = upfront_hardware_capex * (training_time * experiment_factor) / a100_server_lifetime
print_median_and_ci(amortized_server_capex)
print_median_and_ci(amortized_server_capex + 0.01 * total_cost + 0.178 * total_cost)

Median: 5.5e+07 [90% CI: 2.9e+07, 1e+08]
Median: 1.5e+08 [90% CI: 1.2e+08, 2e+08]


In [ ]:
# Yafah's labour cost estimate was based on 1 year of spending, but we're only counting 6 months of training + experiments
adjusted_personnel_cost = (training_time * experiment_factor) * 0.178 * total_cost
adjusted_personnel_cost

46297056.81040383

In [ ]:
cost_data_amortized_capex['GPT-4'] = {
    'GPU servers': amortized_server_capex.mean() * 0.569,
    'Other hardware': amortized_server_capex.mean() * 0.244,
    'Energy': 0.01 * total_cost,
    'Personnel': adjusted_personnel_cost,
}

# Gemini Ultra

See https://colab.research.google.com/drive/1XEKlSo-3DCFp686yGOwwfS6_DEHsFimd#scrollTo=yqWMux2iZL8L

In [ ]:
# Lognormal distribution parameters for project duration
p_5th_project_duration_years = 7 / 12
p_95th_project_duration_years = 20 / 12

experiments_duration = lognorm_from_90_ci(p_5th_project_duration_years, p_95th_project_duration_years, N)

project_duration_hours = experiments_duration * HOURS_PER_YEAR

print_median_and_ci(experiments_duration, ci=[5, 95])

Median: 0.98 [90% CI: 0.57, 1.7]


In [ ]:
chips_per_pod = 4096
number_of_pods = np.random.uniform(12, 19, N)
number_of_chips = number_of_pods * chips_per_pod
print_median_and_ci(number_of_chips)

Median: 6.4e+04 [90% CI: 5e+04, 7.7e+04]


In [ ]:
cost_per_kwh = 0.083  # default assumption for US data centers
power_per_chip = 0.3  # Unsure - TPUv3 is under 300W for most models in https://arxiv.org/pdf/2104.10350.pdf, but this is TPUv4. See also https://cloud.google.com/tpu/docs/v4.
energy_cost = number_of_chips * power_per_chip * project_duration_hours * cost_per_kwh
print_median_and_ci(energy_cost)

Median: 1.3e+07 [90% CI: 7.5e+06, 2.3e+07]


In [ ]:
hardware_cost = 6.7e8
# Use GPU/non-GPU fractions from Yafah Edelman as a placeholder
cost_data_upfront_capex['Gemini Ultra'] = {
    'GPU servers': 0.7 * hardware_cost,
    'Other hardware': 0.3 * hardware_cost,
    'Energy': energy_cost.mean(),
    'Personnel': 1.6e8,
}

In [ ]:
tpuv4_server_lifetime = lognorm_from_90_ci(3, 8, N)  # years

In [ ]:
cost_data_amortized_capex['Gemini Ultra'] = {
    'GPU servers': 0.7 * np.mean(hardware_cost * experiments_duration / tpuv4_server_lifetime),
    'Other hardware': 0.3 * np.mean(hardware_cost * experiments_duration / tpuv4_server_lifetime),
    'Energy': energy_cost.mean(),
    'Personnel': 1.6e8,
}

In [ ]:
cost_data_amortized_capex

{'GPT-3': {'GPU servers': 21101057.56614471,
  'Other hardware': 3738710.870780002,
  'Energy': 213642.0,
  'Personnel': 8326041.174462708},
 'OPT-175B': {'GPU servers': 1251889.3745022374,
  'Other hardware': 395806.9399623841,
  'Energy': 39126.0672,
  'Personnel': 2621423.5046858904},
 'GPT-4': {'GPU servers': 34061173.90082,
  'Other hardware': 14606197.595430722,
  'Energy': 5000000.0,
  'Personnel': 46297056.81040383},
 'Gemini Ultra': {'GPU servers': 104527413.47373964,
  'Other hardware': 44797462.91731699,
  'Energy': 14313389.058145,
  'Personnel': 160000000.0}}

# Plots

In [39]:
cost_component_names.insert(0, 'R&D staff cost')

In [40]:
np.percentile(cost_data['GPT-3 175B (davinci)']['R&D staff cost'], 97.5)

24331637.241142347

In [41]:
component = 'R&D staff cost'
[np.percentile(cost_data[model][component], 97.5) for model in models[:1]]

[24331637.241142347]

In [46]:
# Sample data
categories = ['Category A', 'Category B', 'Category C', 'Category D']
group1_values = [20, 14, 23, 25]
group2_values = [15, 18, 19, 22]

# Absolute error bars for group 1
group1_error_lower = [18, 12, 20, 22]  # Lower bounds
group1_error_upper = [22, 16, 26, 28]  # Upper bounds

# Absolute error bars for group 2
group2_error_lower = [14, 16, 17, 20]  # Lower bounds
group2_error_upper = [16, 20, 21, 24]  # Upper bounds

# Create a grouped bar chart with error bars
fig = go.Figure()

# Group 1
for component in cost_component_names:
    y_values = [np.median(cost_data[model][component]) for model in models[:2]]
    y_lows = [np.percentile(cost_data[model][component], 2.5) for model in models[:2]]
    y_highs = [np.percentile(cost_data[model][component], 97.5) for model in models[:2]]
    print(component)
    print(y_values, y_lows, y_highs)
    fig.add_trace(go.Bar(
        x=models[:1],
        y=y_values,
        error_y=dict(
            type='data',
            symmetric=False,  # Use asymmetric error bars
            array=[y_highs[i] - y_values[i] for i in range(len(y_values))],  # Upper bounds of the error bars
            arrayminus=[y_values[i] - y_lows[i] for i in range(len(y_values))],  # Lower bounds
            visible=True  # Make error bars visible
        ),
        name=component
    ))

# log y axis
fig.update_yaxes(type="log")

# Customize the layout
fig.update_layout(
    barmode='group',  # Group bars together
    title='Total amortized model development costs',
    xaxis=dict(
        title='Model'
    ),
    yaxis=dict(
        title='Cost (2023 USD, log scale)'
    ),
    template='plotly_white'  # Choose a template (optional)
)

# Show the plot
fig.show()


R&D staff cost
[7473965.076211616] [1784993.886001776] [24331637.241142347]
AI accelerator chip cost
[2291881.0206765337] [960833.8873991987] [5457968.581023566]
Other server components cost
[1499862.804680364] [649013.697432466] [3669268.553191867]
Cluster-level interconnect cost
[544418.6871672975] [231516.39255564148] [1228530.5068792806]
Energy cost
[210948.3858575925] [72267.74916077894] [585662.2279505869]


In [ ]:
# Prepare data for Plotly
categories = list(next(iter(cost_data_amortized_capex.values())).keys())
models = [k for k in cost_data_amortized_capex.keys()] # if 'OPT' not in k]

In [ ]:
total_costs = {model:sum([cost_data_amortized_capex[model][category] for category in categories]) for model in models}
total_costs

{'GPT-3': 33379451.611387417,
 'OPT-175B': 4308245.886350512,
 'GPT-4': 99964428.30665456,
 'Gemini Ultra': 323638265.4492016}

In [ ]:
# Creating the figure
fig = go.Figure()

for category in ['GPU servers', 'Other hardware', 'Personnel', 'Energy']:
    fig.add_trace(go.Bar(
        name=category,
        x=list(models),
        y=[cost_data_amortized_capex[model][category] for model in models],
        # text=[human_format(cost_data[model][category]) for model in models],
        # text=[f'{cost_data_amortized_capex[model][category] / total_costs[model] * 100:.1f}%' for model in models],
        textposition='auto',
    ))

# Log y axis
# fig.update_yaxes(type='log')
# fig.update_yaxes(range=[7, 9])

# y ticks
fig.update_yaxes(range=[0, 1e9])
fig.update_yaxes(tickvals=[0, 2e8, 4e8, 6e8, 8e8, 1e9], ticktext=['0', '200M', '400M', '600M', '800M', '1B'])

# Update the layout
fig.update_layout(
    barmode='stack',
    title='Full capital and operational costs of experiments and training',
    xaxis_title='Models',
    yaxis_title='Cost',
    legend_title='Cost Components',
    width=600,
    height=400,
)

# Save plot
save_plot(fig, results_dir, 'full_costs_stacked')

# Show the figure
fig.show()

In [ ]:
# Creating the figure
fig = go.Figure()

for category in ['GPU servers', 'Other hardware', 'Personnel', 'Energy']:
    proportions = [cost_data_amortized_capex[model][category] / total_costs[model] * 100 for model in models]
    fig.add_trace(go.Bar(
        name=category,
        x=list(models),
        y=proportions,
        # text=[human_format(cost_data[model][category]) for model in models],
        # text=[f'{pp:.1f}%' for pp in proportions],
        textposition='auto',
    ))

# y ticks
fig.update_yaxes(range=[0, 100])

# Update the layout
fig.update_layout(
    barmode='stack',
    title='Proportions of cost for experiments and training',
    xaxis_title='Models',
    yaxis_title='Proportion (%)',
    legend_title='Cost Components',
    width=600,
    height=400,
)

# Save plot
save_plot(fig, results_dir, 'cost_proportions_stacked')

# Show the figure
fig.show()